---
**NOTE** \
After installing tensorflow 1.5, you need to restart runtime. You will be asked to do so by clicking button upon installation completion.
This notebook is developed to run on Colab.
---

In [1]:
!pip3 install tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 41kB/s 
     |████████████████████████████████| 51kB 9.5MB/s 
     |████████████████████████████████| 3.8MB 52.2MB/s 
     |████████████████████████████████| 512kB 52.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=7ed6fbbba5e570ea31c55ed396bf98e01949117f8d69105b83be583354c00b41
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tens

In [2]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [3]:
!git clone https://github.com/fakhralwajih/DeepOnKHATT.git

Cloning into 'DeepOnKHATT'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 73 (delta 11), reused 71 (delta 9), pack-reused 0
Unpacking objects: 100% (73/73), done.


In [4]:
# change dir to DeepOnKHATT dir
import os
os.chdir('DeepOnKHATT')

In [5]:
#download pre-trained models
!gdown --id 1-YAltfi_4Klvu_-f72iSkHboM46-iH_t --output lm/trie
!gdown --id 1MqhnAcXMwT_nq_z-01CRhWKLYJYZBa1A --output lm/lm.binary
!gdown --id  1Z_gzzWVjskv_1JqErGuz8ZVfCSNaC3VY --output models/models.zip
!unzip models/models.zip -d models/

Downloading...
From: https://drive.google.com/uc?id=1-YAltfi_4Klvu_-f72iSkHboM46-iH_t
To: /content/DeepOnKHATT/lm/trie
14.0MB [00:00, 86.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1MqhnAcXMwT_nq_z-01CRhWKLYJYZBa1A
To: /content/DeepOnKHATT/lm/lm.binary
1.07GB [00:10, 100MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1Z_gzzWVjskv_1JqErGuz8ZVfCSNaC3VY
To: /content/DeepOnKHATT/models/models.zip
344MB [00:04, 69.1MB/s]
Archive:  models/models.zip
  inflating: models/model.ckpt-14.meta  
  inflating: models/model.ckpt-14.data-00000-of-00001  
  inflating: models/model.ckpt-14.index  


In [6]:
#install decoder 
!pip3 install ds_ctcdecoder==0.6.1

     |████████████████████████████████| 5.6MB 18.4MB/s 


In [7]:
from features.feature import calculate_feature_vector_sequence 
from features.preprocessing import preprocess_handwriting
from rnn import BiRNN as BiRNN_model
from datasets import pad_sequences,sparse_tuple_from ,handwriting_to_input_vector


In [8]:
import argparse
import numpy as np
import tensorflow as tf
from ds_ctcdecoder import ctc_beam_search_decoder, Scorer
from text import Alphabet,get_arabic_letters,decodex

In [9]:
letters_ar=get_arabic_letters()
alphabet = Alphabet('alphabet.txt')
#convert this to funcation
mapping={}
with open('arabic_mapping.txt','r', encoding='utf-8') as inf:
    for line in inf:
        key,val=line.split('\t')
        mapping[key]=val.strip()
mapping[' ']=' '


## imports for writing canvas

In [10]:
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode

In [11]:
from configparser import ConfigParser
config_file='neural_network.ini'
model_path='models/model.ckpt-14'
parser = ConfigParser()


In [12]:
parser.read('neural_network.ini')
config_header='nn'       
network_type = parser.get(config_header , 'network_type')
n_context = parser.getint(config_header, 'n_context') 
n_input = parser.getint(config_header, 'n_input')
beam_search_decoder = parser.get(config_header, 'beam_search_decoder')        


In [13]:
#LM setting
config_header='lm' 
lm_alpha=parser.getfloat(config_header , 'lm_alpha')
lm_beta=parser.getfloat(config_header , 'lm_beta')
beam_width=parser.getint(config_header , 'beam_width')
cutoff_prob=parser.getfloat(config_header , 'cutoff_prob')
cutoff_top_n= parser.getint(config_header , 'cutoff_top_n')



In [14]:
conf_path='neural_network.ini'
input_tensor = tf.placeholder(tf.float32, [None, None, n_input + (2 * n_input * n_context)], name='input')    
seq_length = tf.placeholder(tf.int32, [None], name='seq_length')
logits, summary_op = BiRNN_model(conf_path,input_tensor,tf.to_int64(seq_length),n_input,n_context)
#if you need to try greedy decoder without LM
# decoded, log_prob = ctc_ops.ctc_greedy_decoder(logits, seq_length, merge_repeated=True)

Instructions for updating:
Use `tf.cast` instead.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer ins

In [15]:
lm_binary_path='lm/lm.binary'
lm_trie_path='lm/trie'

scorer=None
scorer = Scorer(lm_alpha, lm_beta,lm_binary_path, lm_trie_path,alphabet)


In [16]:
config_file='neural_network.ini'

In [17]:
saver = tf.train.Saver()
# create the session
sess = tf.Session()
saver.restore(sess, 'models/model.ckpt-14')
print('Model restored') 

INFO:tensorflow:Restoring parameters from models/model.ckpt-14
Model restored


In [19]:
canvas_html = """
<canvas id="mycanvas" width=%d height=%d style="border:1px solid #000000;"></canvas>
 <br />
<button>Recoginize</button>

<script>
var canvas = document.getElementById('mycanvas')
var ctx = canvas.getContext('2d')
ctx.lineWidth = %d
ctx.canvas.style.touchAction = "none";
var button = document.querySelector('button')
var mouse = {x: 0, y: 0}
var points=[]

canvas.addEventListener('pointermove', function(e) {
  mouse.x = e.pageX - this.offsetLeft
  mouse.y = e.pageY - this.offsetTop
})
canvas.onpointerdown = ()=>{
  ctx.beginPath()
  ctx.moveTo(mouse.x, mouse.y)
  
  canvas.addEventListener('pointermove', onPaint)
}
canvas.onpointerup = ()=>{
  canvas.removeEventListener('pointermove', onPaint)
  points.pop()
  points.push([mouse.x,mouse.y,1])
}
var onPaint = ()=>{
  ctx.lineTo(mouse.x, mouse.y)
  ctx.stroke()
  points.push([mouse.x,mouse.y,0])
}
var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve(canvas.toDataURL('image/png'))
  }
})
</script>
"""
def draw(filename='drawing.png', w=900, h=200, line_width=1):
  display(HTML(canvas_html % (w, h, line_width)))
  data = eval_js("data")
  points=eval_js("points")
  # strokes = Utils.Rearrange(strokes, 20);
  points=np.array(points)

  # points=rearrange(points)


  # print("Points before pre",points.shape)
  NORM_ARGS = ["origin","filp_h","smooth", "slope", "resample", "slant", "height"]
  FEAT_ARGS = ["x_cor","y_cor","penup","dir", "curv", "vic_aspect", "vic_curl", "vic_line", "vic_slope", "bitmap"]
  # print("Normalizing trajectory...")
  traj = preprocess_handwriting(points, NORM_ARGS)
  # print(traj)
  # print("Calculating feature vector sequence...")
  feat_seq_mat = calculate_feature_vector_sequence(traj, FEAT_ARGS)
  feat_seq_mat=feat_seq_mat.astype('float32')
  feat_seq_mat.shape

  data = []

  train_input=handwriting_to_input_vector(feat_seq_mat,20,9)
  train_input = train_input.astype('float32')

  data.append(train_input)
  # data_len

  data = np.asarray(data)
  # data_len = np.asarray(train_input)


  # Pad input to max_time_step of this batch
  source, source_lengths = pad_sequences(data)
  my_logits=sess.run(logits, feed_dict={
                  input_tensor: source,                    
                  seq_length: source_lengths}
              )
  my_logits = np.squeeze(my_logits)
  maxT, _ = my_logits.shape # dim0=t, dim1=c
	
            # apply softmax
  res = np.zeros(my_logits.shape)
  for t in range(maxT):
      y = my_logits[t, :]
      e = np.exp(y)
      s = np.sum(e)
      res[t, :] = e / s
            
  decoded = ctc_beam_search_decoder(res, alphabet, beam_width,
                                  scorer=scorer, cutoff_prob=cutoff_prob,
                                  cutoff_top_n=cutoff_top_n)

  print("Result : "+decodex(decoded[0][1],mapping))
  

**Note**  \
The next cell, you can run multiple times as you wish. You may want to write many samples. \
All the above cells should run only once. \

Run the cell and write on the canvas while the cell is running. Then, click on "Recoginize" button to get the recognition result.


In [24]:
draw()

Result : الله


The above canvas was tested on PC mouse and S pen (samsung tablet) and works well with S pen.